## Model을 만들기 전에 여러가지 기법을 이용하여 test를 해보았습니다. 

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm   #SVM!!!
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import normalize

train_data = pd.read_csv('C:/Users/user/ML Term/train_data.csv')
test_data = pd.read_csv('C:/Users/user/ML Term/test_data.csv')

# 데이터 전처리

#### 데이터 처리는 one hot encoding 하고 머신러닝 기법에 따라 normalize 하는게 좋다

# One Hot Encoding

In [ ]:
train_data_ohe = pd.get_dummies(train_data.iloc[:,:-1])  #protocol 성분을 onehotencoding
test_data_ohe = pd.get_dummies(test_data.iloc[:,1:-1]) 

In [ ]:
X = train_data_ohe
y = train_data.iloc[:,-1]

In [ ]:
test_X = test_data_ohe
test_y = test_data.iloc[:,-1]

In [ ]:
print(X.shape)
print(test_X.shape)

In [ ]:
X_train_cv, X_test_cv, y_train_cv, y_test_cv = train_test_split(X, y, test_size=0.4, random_state=0) #Cross Validation 용

# Data Selection

### confusion matrix를 확인한 결과 class r2l, u2r의 데이터가 너무 적고 normal데이터가 너무 많은 관계로 잘 classify가 되지 않는다.  따라서 data를 최대한 균등하게 조절하여 train해보자

In [ ]:
EX = pd.DataFrame(data=train_data, columns=['xAttack'])

In [ ]:
xAttack = EX['xAttack']
xAttack[0]

In [ ]:
#dos,normal,probe,r2l,u2r
dos_count = 0
dos_array = []
normal_count = 0
normal_array = []
probe_count = 0
probe_array = []
r2l_count = 0
r2l_array = []
u2r_count = 0
u2r_array = []
for i in range(0,len(xAttack)):
    if(xAttack[i]=='dos'):
        dos_array.append(i)
        
for i in range(0,len(xAttack)):
    if(xAttack[i]=='normal'):
        normal_array.append(i)
        
for i in range(0,len(xAttack)):
    if(xAttack[i]=='probe'):
        probe_array.append(i)
        
for i in range(0,len(xAttack)):
    if(xAttack[i]=='r2l'):
        r2l_array.append(i)
        
for i in range(0,len(xAttack)):
    if(xAttack[i]=='u2r'):
        u2r_array.append(i)

In [ ]:
print(len(dos_array))
print(len(normal_array))
print(len(probe_array))
print(len(r2l_array))
print(len(u2r_array))

In [ ]:
train_data_fs = train_data_ohe.copy()

### 사용하고 싶은 데이터 수 -> exdata

In [ ]:
exdata = 800  #사용 하고 싶은 데이터
train_data_fs_dos = pd.DataFrame(data=train_data_fs, index=dos_array[0:exdata])
train_data_fs_normal = pd.DataFrame(data=train_data_fs, index=normal_array[0:exdata])
train_data_fs_probe = pd.DataFrame(data=train_data_fs, index=probe_array[0:exdata])
train_data_fs_r2l = pd.DataFrame(data=train_data_fs, index=r2l_array)
train_data_fs_u2r = pd.DataFrame(data=train_data_fs, index=u2r_array)

test_data_fs_dos = pd.DataFrame(data=y, index=dos_array[0:exdata])
test_data_fs_normal = pd.DataFrame(data=y, index=normal_array[0:exdata])
test_data_fs_probe = pd.DataFrame(data=y, index=probe_array[0:exdata])
test_data_fs_r2l = pd.DataFrame(data=y, index=r2l_array)
test_data_fs_u2r = pd.DataFrame(data=y, index=u2r_array)

In [ ]:
train_data_fs_sum = train_data_fs_dos.append(train_data_fs_normal).append(train_data_fs_probe).append(train_data_fs_r2l).append(train_data_fs_u2r)
X_ex = train_data_fs_sum

test_data_fs_sum = test_data_fs_dos.append(test_data_fs_normal).append(test_data_fs_probe).append(test_data_fs_r2l).append(test_data_fs_u2r)
y_ex = test_data_fs_sum

In [ ]:
type(y_ex) # frame 을 series로 바꿔줘야 한다

In [ ]:
y_ex = y_ex.iloc[:,0]

In [ ]:
from sklearn import tree
decision_tree = tree.DecisionTreeClassifier(criterion='entropy', max_depth=10, random_state=0)
decision_tree.fit(X_ex, y_ex) 
from sklearn.metrics import accuracy_score

y_pred_tree = decision_tree.predict(test_X)
print('Accuracy: %.3f' % accuracy_score(test_y, y_pred_tree))

# Test 용 application
### data를 얼마나 제거 했을때 + feature를 몇개만 사용했을 때 

### 성능이 좋을지 test하기 위해 간단한 application 작성

### DT에 대해 test data만 봤을때는 데이터 800개 + feature 35개 사용했을때 성능이 좋았다. -> 81.6%

In [ ]:
from sklearn import tree
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

decision_tree = tree.DecisionTreeClassifier(criterion='entropy', max_depth=10, random_state=0)

for exdata in range(1, 200):
    exdata = exdata * 50
    train_data_fs_dos = pd.DataFrame(data=train_data_fs, index=dos_array[0:exdata])
    train_data_fs_normal = pd.DataFrame(data=train_data_fs, index=normal_array[0:exdata])
    train_data_fs_probe = pd.DataFrame(data=train_data_fs, index=probe_array)
    train_data_fs_r2l = pd.DataFrame(data=train_data_fs, index=r2l_array)
    train_data_fs_u2r = pd.DataFrame(data=train_data_fs, index=u2r_array)

    test_data_fs_dos = pd.DataFrame(data=y, index=dos_array[0:exdata])
    test_data_fs_normal = pd.DataFrame(data=y, index=normal_array[0:exdata])
    test_data_fs_probe = pd.DataFrame(data=y, index=probe_array)
    test_data_fs_r2l = pd.DataFrame(data=y, index=r2l_array)
    test_data_fs_u2r = pd.DataFrame(data=y, index=u2r_array)


    train_data_fs_sum = train_data_fs_dos.append(train_data_fs_normal).append(train_data_fs_probe).append(train_data_fs_r2l).append(train_data_fs_u2r)
    X_ex = train_data_fs_sum

    test_data_fs_sum = test_data_fs_dos.append(test_data_fs_normal).append(test_data_fs_probe).append(test_data_fs_r2l).append(test_data_fs_u2r)
    y_ex = test_data_fs_sum

    type(y_ex) # frame 을 series로 바꿔줘야 한다

    y_ex = y_ex.iloc[:,0]

    
    decision_tree.fit(X_ex, y_ex) 
    from sklearn.metrics import accuracy_score

    y_pred_tree = decision_tree.predict(test_X)
    if (accuracy_score(test_y, y_pred_tree) > 0.78):
        print('사용한 data ', exdata)
        print('Accuracy: %.3f' % accuracy_score(test_y, y_pred_tree))

        for best in range(5,41):
            KBest = SelectKBest(chi2, k=best)
            KBest = KBest.fit(X_ex,y_ex)
            X_fs = KBest.transform(X_ex)
            test_X_fs = KBest.transform(test_X)

            model_fs = decision_tree.fit(X_fs, y_ex)

            y_pred_erf = model_fs.predict(test_X_fs)
            if(accuracy_score(test_y, y_pred_erf) > 0.8):
                print('  KBest수 ',best)
                print('  FS 하고 Accuracy: %.3f' % accuracy_score(test_y, y_pred_erf))
print("끝!")

# Normalization

In [ ]:
X_n = normalize(X, norm='l1')   #one hot encoding 을 해줘야 할듯!!!
test_X_n = normalize(test_X, norm='l1')   #l1, l2 두 종류가 있다!
X_ex_n = normalize(X_ex, norm='l1')

# Feature Selection

# RFE 
http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html#sklearn.feature_selection.RFE

Wrapper Methods

In [ ]:
# Recursive Feature Elimination
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

from sklearn import tree
model = tree.DecisionTreeClassifier()

#model = LogisticRegression()
# create the RFE model and select 3 attributes
rfe = RFE(model)       #전체는 41! 35attribute 선택
rfe = rfe.fit(X_ex, y_ex)
# summarize the selection of the attributes
print(rfe.support_)
print(rfe.ranking_)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred_rfe = rfe.predict(test_X)
print('Accuracy: %.3f' % accuracy_score(test_y, y_pred_rfe))

ranking에 '1'로 표시 되는 feature들을 골라서 training한다. 

In [ ]:
# Recursive Feature Elimination
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

# from sklearn.neighbors import KNeighborsClassifier

#model = KNeighborsClassifier()
model = RandomForestClassifier()

rfe = RFE(model)       #전체는 40! dafault일때 자동을 선택??
rfe = rfe.fit(X_ex, y_ex)
# summarize the selection of the attributes
print(rfe.support_)
print(rfe.ranking_)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred_rfe = rfe.predict(test_X)
print('Accuracy: %.3f' % accuracy_score(test_y, y_pred_rfe))

### rfe를 제외한 나머지는 predict를 지원하지 않는다.... 따라서 따로 test_X를 가공 해야한다.

# Variance Threshold
http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.VarianceThreshold.html#sklearn.feature_selection.VarianceThreshold

normalize data 는 사용 불가

In [ ]:
#예시
from sklearn.feature_selection import VarianceThreshold
sample = [[0, 0, 1], [0, 1, 0], [1, 0, 0], [0, 1, 1], [0, 1, 0], [0, 1, 1]]
vt = VarianceThreshold(threshold=(.8 * (1 - .8)))
vt.fit(sample)
vt.fit_transform(sample)

In [ ]:
from sklearn.feature_selection import VarianceThreshold

vt = VarianceThreshold(threshold=(.8 * (1 - .8)))
vt = vt.fit(X_ex,y_ex)
print(vt.get_support())
print(vt.transform(X_ex).shape)


In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
X_fs = vt.transform(X_ex)
test_X_fs = vt.transform(test_X)

model_original = ExtraTreesClassifier().fit(X_ex, y_ex)
model_fs = ExtraTreesClassifier().fit(X_fs, y_ex)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred_erf = model_original.predict(test_X)
print('FS 하기 전 Accuracy: %.3f' % accuracy_score(test_y, y_pred_erf))

In [ ]:
from sklearn.metrics import accuracy_score

y_pred_erf = model_fs.predict(test_X_fs)
print('FS 하고 Accuracy: %.3f' % accuracy_score(test_y, y_pred_erf))

# Select KBest
http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest

Filter method

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

KBest = SelectKBest(chi2, k=35)
KBest = KBest.fit(X_ex,y_ex)
print(KBest.get_support())
print(KBest.transform(X_ex).shape)

상위 K 개의 feature를 고른다.

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
X_fs = KBest.transform(X_ex)
test_X_fs = KBest.transform(test_X)

model_original = ExtraTreesClassifier().fit(X_ex, y_ex)
model_fs = ExtraTreesClassifier().fit(X_fs, y_ex)

모델은 아무거나 사용해도 괜찮다. 성능이 계속 잘 나오는 random forest를 사용해 보자.

In [ ]:
from sklearn.metrics import accuracy_score

y_pred_erf = model_original.predict(test_X)
print('FS 하기 전 Accuracy: %.3f' % accuracy_score(test_y, y_pred_erf))

In [ ]:
from sklearn.metrics import accuracy_score

y_pred_erf = model_fs.predict(test_X_fs)
print('FS 하고 Accuracy: %.3f' % accuracy_score(test_y, y_pred_erf))

### Test용 application
KBest를 몇개로 했을때 성능이 좋은지 test해보기 위해 간단한 application 제작

기법은 Decision Tree를 사용했다

In [ ]:
from sklearn import tree
decision_tree = tree.DecisionTreeClassifier(criterion='entropy', max_depth=10, random_state=0)

for best in range(5,41):
    KBest = SelectKBest(chi2, k=best)
    KBest = KBest.fit(X_ex,y_ex)
    X_fs = KBest.transform(X_ex)
    test_X_fs = KBest.transform(test_X)

    model_fs = decision_tree.fit(X_fs, y_ex)

    y_pred_erf = model_fs.predict(test_X_fs)
    if(accuracy_score(test_y, y_pred_erf) > 0.78):
        print('KBes수 ',best)
        print('FS 하고 Accuracy: %.3f' % accuracy_score(test_y, y_pred_erf))

### 10~15, 27~40 의 feature 를 선택 했을때 성능이 좋았다. 

# Select from Model
http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectFromModel.html#sklearn.feature_selection.SelectFromModel

Embedded Method

Meta-transformer for selecting features based on importance weights.

In [ ]:
from sklearn import tree
decision_tree = tree.DecisionTreeClassifier(criterion='entropy', max_depth=10, random_state=0)
from sklearn.feature_selection import SelectFromModel

tree_original = decision_tree.fit(X_ex, y_ex)
tree = SelectFromModel(tree_original).fit(X_ex,y_ex)

print(tree.get_support())
print(tree.transform(X_ex).shape)


In [ ]:
X_fs = tree.transform(X_ex)
test_X_fs = tree.transform(test_X)

tree_fs = decision_tree.fit(X_ex, y_ex)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred_lsvc = tree_original.predict(test_X)
print('FS 하기 전 Accuracy: %.3f' % accuracy_score(test_y, y_pred_lsvc))

In [ ]:
from sklearn.metrics import accuracy_score

y_pred_lsvc = tree_fs.predict(test_X)
print('FS 하고 Accuracy: %.3f' % accuracy_score(test_y, y_pred_lsvc))

In [ ]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.feature_selection import SelectFromModel

erf_original = RandomForestClassifier(n_estimators=10, criterion='gini', max_depth=10, max_features="auto").fit(X_ex,y_ex)
erf = SelectFromModel(erf_original).fit(X_ex,y_ex)

print(erf.get_support())
print(erf.transform(X_ex).shape)

In [ ]:
X_fs = erf.transform(X_ex)
test_X_fs = erf.transform(test_X)

erf_fs = RandomForestClassifier(n_estimators=10, criterion='gini', max_depth=10, max_features="auto").fit(X_ex, y_ex)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred_erf = erf_original.predict(test_X)
print('FS 하기 전 Accuracy: %.3f' % accuracy_score(test_y, y_pred_erf))

In [ ]:
from sklearn.metrics import accuracy_score

y_pred_erf = erf_fs.predict(test_X)
print('FS 하고 Accuracy: %.3f' % accuracy_score(test_y, y_pred_erf))

### Feature Selection은 KBest로 직접 상위 몇개를 사용 할지 정할 때 성능이 가장 좋다.

# Classify 방법들 

여기서 성능을 비교하여 Ensemble, feature selection 등등에 사용하자

# Decision Tree

In [ ]:
from sklearn import tree
decision_tree = tree.DecisionTreeClassifier(criterion='entropy', max_depth=10, random_state=0)

In [ ]:
decision_tree.fit(X_ex, y_ex) 

In [ ]:
cross_val_score(decision_tree, X_test_cv, y_test_cv, cv=10)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred_tree = decision_tree.predict(test_X)
print('Accuracy: %.3f' % accuracy_score(test_y, y_pred_tree))

labeling 만 했을 때 보다 one hot encoding 하였을 때 성능 조금 향상 

L1 은 0.760, L2 normalize 시 0.76 -> 0.727 로 성능이 떨어졌다....

decision tree는 normalize를 하지 않는것이 더 좋다. 

# Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

In [ ]:
gnb.fit(X_n,y)

In [ ]:
cross_val_score(gnb, X_test_cv, y_test_cv, cv=10)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred_gnb = gnb.predict(test_X_n)
print('Accuracy: %.3f' % accuracy_score(test_y, y_pred_gnb))

In [ ]:
y_pred_gnb2 = gnb.fit(X_n,y).predict(X)
print("Number of mislabeled points out of a total %d points : %d"% (X.shape[0],(y != y_pred_gnb2).sum()))

normalize 시  0.289 -> 0.489 로 상승 하지만 그래도 성능이 너무 낮아 사용할 수 없다 L1: 0.480

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [ ]:
lr.fit(X_n,y)

In [ ]:
cross_val_score(lr, X_test_cv, y_test_cv, cv=10)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred_lr = lr.predict(test_X_n)
print('Accuracy: %.3f' % accuracy_score(test_y, y_pred_lr))

one hot encoding 한 뒤 성능 소폭 상승 0.614 -> 0.619, normalize한 뒤 성능이 L2 0.619 -> 0.641로 증가함, L1옵션은 성능이 0.643

# Perceptron

In [ ]:
from sklearn.linear_model import Perceptron
p = Perceptron()

In [ ]:
p.fit(X_n,y)

In [ ]:
cross_val_score(p, X_test_cv, y_test_cv, cv=10)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred_p = p.predict(test_X_n)
print('Accuracy: %.3f' % accuracy_score(test_y, y_pred_p))

normalize L2 한 뒤 성능 대폭 향상 0.119 -> 0.649, L1 옵셥시 0.662로 다시 상승

# MLP

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(activation='relu', hidden_layer_sizes=(5,5,5))

In [ ]:
mlp.fit(X_ex,y_ex)

In [ ]:
cross_val_score(mlp, X_test_cv, y_test_cv, cv=10)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred_mlp = mlp.predict(test_X)
print('Accuracy: %.3f' % accuracy_score(test_y, y_pred_mlp))

one hot encoding을 하고 성능 0.699 -> 0.729 으로 향상, normalize 하고 나서는 어째선지 성능이 0.729 ->  0.748로
MLP는 학습 할 때마다 성능이 계속 바뀐다.

시간이 너무 오래걸려 사용하기 어렵다

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()

In [ ]:
knn.fit(X_ex,y_ex)

In [ ]:
cross_val_score(knn, X_test_cv, y_test_cv, cv=10)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred_knn = knn.predict(test_X)
print('Accuracy: %.3f' % accuracy_score(test_y, y_pred_knn))

one hot encoding한 뒤 0.736 -> 0.735 normalize 한 뒤 0.735 -> 0.745, L1 시 0.743

성능은 괜찮지만 Decision Tree가 더 좋다

# SVM

### SVM 코드들은 Data Selection으로 데이터 양을 확 줄인 상태가 아니라면 
### Training 시간이 너무 오래 걸려 사용 할 수 없다

In [ ]:
from sklearn import svm   #SVM은 너무 오래 걸린다.... 자기전에 켜두고 자자
clf = svm.SVC()

In [ ]:
clf.fit(X_fs, y_fs)    

In [ ]:
cross_val_score(clf, X_test_cv, y_test_cv, cv=10)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred_clf = clf.predict(test_X)
print('Accuracy: %.3f' % accuracy_score(test_y, y_pred_clf))

In [ ]:
clf2 = svm.SVC(decision_function_shape='ovo')
clf2.fit(X_fs, y_fs) 

In [ ]:
from sklearn.metrics import accuracy_score

y_pred_clf2 = clf2.predict(test_X)
print('Accuracy: %.3f' % accuracy_score(test_y, y_pred_clf2))

In [ ]:
clf3 = svm.SVC(decision_function_shape='ovr')
clf3.fit(X_fs,y_fs)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred_clf3 = clf3.predict(test_X_n)
print('Accuracy: %.3f' % accuracy_score(test_y, y_pred_clf3))

In [ ]:
lin_clf = svm.LinearSVC()
lin_clf.fit(X_fs, y_fs) 

In [ ]:
from sklearn.metrics import accuracy_score

y_pred_lin_clf = lin_clf.predict(test_X)
print('Accuracy: %.3f' % accuracy_score(test_y, y_pred_lin_clf))

In [ ]:
clfr = svm.SVR()
clfr.fit(X_fs, y_fs)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred_clfr = clfr.predict(test_X_n)
print('Accuracy: %.3f' % accuracy_score(test_y, y_pred_clfr))

In [ ]:
linear_svc = svm.SVC(kernel='linear')
linear_svc.kernel
linear_svc.fit(X_fs,y_fs)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred_linear_svc = linear_svc.predict(test_X_n)
print('Accuracy: %.3f' % accuracy_score(test_y, y_pred_linear_svc))

In [ ]:
rbf_svc = svm.SVC(kernel='rbf')
rbf_svc.kernel
rbf_svc.fit(X_n,y)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred_rbf_svc = rbf_svc.predict(test_X_n)
print('Accuracy: %.3f' % accuracy_score(test_y, y_pred_rbf_svc))

### 사용할 만한 기법은 decision tree, MLP, KNN 이다. 

### SVM은 성능이 70프로대가 나오긴하나 시간이 너무 오래 걸려 사용할 수 없다.

# Bagging(KNN)

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
bagging_knn = BaggingClassifier(KNeighborsClassifier(), max_samples=0.5, max_features=0.5)

#bagging with KNN

In [ ]:
bagging_knn.fit(X_ex, y_ex)    

In [ ]:
cross_val_score(bagging_knn, X_test_cv, y_test_cv, cv=10)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred_bagging_knn = bagging_knn.predict(test_X)
print('Accuracy: %.3f' % accuracy_score(test_y, y_pred_bagging_knn))

one hot encoding 후 0.743 -> 0.749, normalize 후 0.749 -> 0.737 로 성능이 떨어진게 아니라 다시 0.747!! MLP와 마찬가지로 측정할때 마다 값이 왔다갔다 함....

# Bagging(DT)

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
decision_tree = DecisionTreeClassifier(criterion='entropy', max_depth=10, random_state=0)
bagging_knn = BaggingClassifier(decision_tree, max_samples=1.0, max_features=1.0)

In [ ]:
bagging_knn.fit(X_ex, y_ex)    

In [ ]:
from sklearn.metrics import accuracy_score

y_pred_bagging_knn = bagging_knn.predict(test_X)
print('Accuracy: %.3f' % accuracy_score(test_y, y_pred_bagging_knn))

### Bagging은 Decision Tree로 시도 했을때 성능이 가장 좋았다

# AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ab = AdaBoostClassifier(n_estimators=100)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
ab = AdaBoostClassifier(DecisionTreeClassifier(criterion='entropy', max_depth=10, random_state=0), n_estimators=100, learning_rate=1)
ab.fit(X_ex, y_ex)    

In [ ]:
cross_val_score(ab, X_test_cv, y_test_cv, cv=10)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred_ab = ab.predict(test_X)
print('Accuracy: %.3f' % accuracy_score(test_y, y_pred_ab))

### AdaBoost도 Decision tree를 이용할 때 성능이 좋았다

# Hard Voting 예시

In [ ]:
#Hard voting
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.ensemble import VotingClassifier

h1 = KNeighborsClassifier()
h2 = RandomForestClassifier(random_state=1)
h3 = MLPClassifier()

hvote = VotingClassifier(estimators=[('knn', h1), ('rf', h2), ('mlp', h3)], voting='hard')

for clf, label in zip([h1, h2, h3, hvote], ['KNN', 'Random Forest', 'MLP', 'Ensemble']):
    scores = cross_val_score(clf, X, y, cv=5, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

In [ ]:
hvote.fit(X,y)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred_hvote = hvote.predict(test_X)
print('Accuracy: %.3f' % accuracy_score(test_y, y_pred_hvote))

# Soft Voting 예시

In [ ]:
#Soft Voting
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

from itertools import product
from sklearn.ensemble import VotingClassifier

# Training classifiers
s1 = KNeighborsClassifier()
s2 = RandomForestClassifier(random_state=1)
s3 = MLPClassifier()

svote = VotingClassifier(estimators=[('dt', s1), ('knn', s2), ('svc', s3)], voting='soft', weights=[2,1,2]) #여기서 weight를 준다. 

s1.fit(X,y)
s2.fit(X,y)
s3.fit(X,y)
svote.fit(X,y)

### voting은 Hard Voting만 사용했다.

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth=10, max_features="auto") 
# n_estimator가 tree의 크기

In [ ]:
rf.fit(X_ex, y_ex)

In [ ]:
cross_val_score(rf, X_test_cv, y_test_cv, cv=10)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred_rf = rf.predict(test_X)
print('Accuracy: %.3f' % accuracy_score(test_y, y_pred_rf))

# Extra RF

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
erf = ExtraTreesClassifier(n_estimators=1000, criterion='entropy', max_depth=10, max_features="auto")

In [ ]:
erf.fit(X_ex,y_ex)

In [ ]:
cross_val_score(erf, X_test_cv, y_test_cv, cv=10)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred_rf = erf.predict(test_X)
print('Accuracy: %.2f' % accuracy_score(test_y, y_pred_rf))

### 성능이 보통 Random Forest 보다 Extra tree classifier가 더 잘 나온다

# Random Forest 를 활용한 그래프 분석

In [ ]:
from sklearn_evaluation import plot
%matplotlib inline
plot.feature_importances(rf, top_n=5)
rf.feature_importances_

In [ ]:
plot.confusion_matrix(test_y, rf.predict(test_X)) 

classify한 형태를 보면 class 2 가 다른 데이터에 비해 압도적으로 많아서?? 대부분 classify가 class 2로 쏠리는 것을 볼 수 있다. 
그리고 class r2l, u2r 은 normal과 data가 거의 유사한지 잘 detect를 하지 못한다. 

-> class 분포를 조금 균등하게 해서 classify 해봐야겠다.

In [ ]:
plot.precision_recall(test_y, rf.predict_proba(test_X))

In [ ]:
plot.roc(test_y, rf.predict_proba(test_X))

In [ ]:
from sklearn.model_selection import learning_curve, validation_curve
train_sizes, train_scores, test_scores = learning_curve(rf, test_X, test_y)

In [ ]:
plot.learning_curve(train_scores, test_scores, train_sizes)

In [ ]:
param_range = np.array([1, 10, 100])
param_name = "n_estimators"
train_scores, test_scores = validation_curve(rf, test_X, test_y, param_name=param_name, param_range=param_range)

In [ ]:
plot.validation_curve(train_scores, test_scores, param_range, param_name)

# 최종 모델!!

### 데이터 불러오기

In [31]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm   #SVM!!!
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import normalize

train_data = pd.read_csv('C:/Users/user/ML Term/train_data.csv')  #training에 사용할 데이터
test_data = pd.read_csv('C:/Users/user/ML Term/test_data.csv')   #test에 사용할 데이터

### one hot encoding

In [32]:
train_data_ohe = pd.get_dummies(train_data.iloc[:,:-1])  #protocol 성분을 onehotencoding
test_data_ohe = pd.get_dummies(test_data.iloc[:,1:-1]) 

X = train_data_ohe
y = train_data.iloc[:,-1]

test_X = test_data_ohe
test_y = test_data.iloc[:,-1]

### Data Extraction

45927
67343
11656
995
52

class 별 원래 data 수

In [33]:
EX = pd.DataFrame(data=train_data, columns=['xAttack'])
xAttack = EX['xAttack']
#dos,normal,probe,r2l,u2r
dos_count = 0
dos_array = []
normal_count = 0
normal_array = []
probe_count = 0
probe_array = []
r2l_count = 0
r2l_array = []
u2r_count = 0
u2r_array = []
for i in range(0,len(xAttack)):
    if(xAttack[i]=='dos'):
        dos_array.append(i)
        
for i in range(0,len(xAttack)):
    if(xAttack[i]=='normal'):
        normal_array.append(i)
        
for i in range(0,len(xAttack)):
    if(xAttack[i]=='probe'):
        probe_array.append(i)
        
for i in range(0,len(xAttack)):
    if(xAttack[i]=='r2l'):
        r2l_array.append(i)
        
for i in range(0,len(xAttack)):
    if(xAttack[i]=='u2r'):
        u2r_array.append(i)
        
train_data_fs = train_data_ohe.copy()

exdata = 800  #사용 하고 싶은 데이터양

train_data_fs_dos = pd.DataFrame(data=train_data_fs, index=dos_array[0:exdata])
train_data_fs_normal = pd.DataFrame(data=train_data_fs, index=normal_array[0:exdata])
train_data_fs_probe = pd.DataFrame(data=train_data_fs, index=probe_array[0:exdata])
train_data_fs_r2l = pd.DataFrame(data=train_data_fs, index=r2l_array[0:exdata])
train_data_fs_u2r = pd.DataFrame(data=train_data_fs, index=u2r_array)

test_data_fs_dos = pd.DataFrame(data=y, index=dos_array[0:exdata])
test_data_fs_normal = pd.DataFrame(data=y, index=normal_array[0:exdata])
test_data_fs_probe = pd.DataFrame(data=y, index=probe_array[0:exdata])
test_data_fs_r2l = pd.DataFrame(data=y, index=r2l_array[0:exdata])
test_data_fs_u2r = pd.DataFrame(data=y, index=u2r_array)

train_data_fs_sum = train_data_fs_dos.append(train_data_fs_normal).append(train_data_fs_probe).append(train_data_fs_r2l).append(train_data_fs_u2r)
X_ex = train_data_fs_sum

test_data_fs_sum = test_data_fs_dos.append(test_data_fs_normal).append(test_data_fs_probe).append(test_data_fs_r2l).append(test_data_fs_u2r)
y_ex = test_data_fs_sum

type(y_ex) # frame 을 series로 바꿔줘야 한다
y_ex = y_ex.iloc[:,0]

### Feature selection

### Filter method

In [34]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

KBest = SelectKBest(chi2, k=35)
KBest = KBest.fit(X_ex,y_ex)
print(KBest.get_support())
print(KBest.transform(X_ex).shape)

X_fs = KBest.transform(X_ex)
test_X_fs = KBest.transform(test_X)


[ True  True  True  True  True False  True False  True False  True  True
  True False  True  True  True False False False  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True False  True  True]
(3252, 35)


### Voting 사용

### Bagging(DT)

In [35]:
from sklearn.metrics import accuracy_score

from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
decision_tree = DecisionTreeClassifier(criterion='entropy', max_depth=10, random_state=0)
bagging = BaggingClassifier(decision_tree, max_samples=1.0, max_features=1.0)

bagging.fit(X_fs, y_ex)    
y_pred_bagging = bagging.predict(test_X_fs)
print('Bagging Accuracy: %.3f' % accuracy_score(test_y, y_pred_bagging))

Bagging Accuracy: 0.793


### AdaBoost(DT) 최종 모델에는 사용 X

In [36]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
ab = AdaBoostClassifier(n_estimators=100)
ab = AdaBoostClassifier(DecisionTreeClassifier(criterion='entropy', max_depth=10, random_state=0), n_estimators=100, learning_rate=1)

ab.fit(X_fs, y_ex)  
y_pred_ab = ab.predict(test_X_fs)
print('AdaBoost Accuracy: %.3f' % accuracy_score(test_y, y_pred_ab))

AdaBoost Accuracy: 0.777


### 기본 Decision Tree

In [37]:
from sklearn import tree
decision_tree = tree.DecisionTreeClassifier(criterion='entropy', max_depth=10, random_state=0)

decision_tree.fit(X_fs, y_ex) 
from sklearn.metrics import accuracy_score

y_pred_tree = decision_tree.predict(test_X_fs)
print('Decision_tree Accuracy: %.3f' % accuracy_score(test_y, y_pred_tree))

Decision_tree Accuracy: 0.817


### Extra Random Forest

In [38]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
#rf = RandomForestClassifier(n_estimators=10, criterion='entropy', max_depth=10, max_features="auto") 
erf = ExtraTreesClassifier(n_estimators=500, criterion='entropy', max_depth=10, max_features="auto") 
# n_estimator가 tree의 크기

erf.fit(X_fs, y_ex)
y_pred_erf = erf.predict(test_X_fs)
print('Extra Accuracy: %.3f' % accuracy_score(test_y, y_pred_erf))

Extra Accuracy: 0.789


In [39]:
from sklearn.ensemble import VotingClassifier


hvote = VotingClassifier(estimators=[('bagging', bagging), ('decision_tree', decision_tree), ('extra random forest', erf)], voting='hard')

# 각각의 Cross validation 
# for clf, label in zip([bagging, ab, erf, hvote], ['bagging', 'adaboost', 'extra random forest', 'Ensemble']):
#     scores = cross_val_score(clf, X, y, cv=5, scoring='accuracy')
#     print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))


In [40]:
hvote.fit(X_fs,y_ex)
y_pred_hvote = hvote.predict(test_X_fs)
print('Voting 했을 때 Accuracy: %.3f' % accuracy_score(test_y, y_pred_hvote))

Voting 했을 때 Accuracy: 0.812


### 가장 성능이 잘 나왔던 Decision Tree를 Bagging 한 것과 기본 Decision Tree,  Extra Tree Classifier 중에 Voting을 하여 최종 classify를 하는 모델을 만들었습니다. 

### Feature Selection 과 Data Selection은 Decision Tree의 결과가 가장 좋았던 수치를 기준으로 수행 하였습니다. 
### 과제에서 주어진 Test Data로 성능을 측정 했을때 
### Decision Tree 단일 모델이 81.7%이었고, 최종 모델로 Voting 했을때 79% ~ 82% 의 성능을 보여주었습니다. 